In [1]:
#=
2024-05-15
演示 阿雷拉诺 第二章 的 几个公式。主要涉及 乔列斯基分解 以及 前向后向 离差
创建于2024-05-15。修改于2024-06-11。中间有两个改进：1、在.ipynb文件中引入markdown，2、修改注释的字体。不要斜体，颜色不要太浅。
2024-06-11重新发现在.ipynb中弃用markdown的原因。在.ipynb中markdown的编辑和显示是两种状态，需要切换，不如.ipynb代码块。是一种状态，可直接阅读和修改。
=#

In [2]:
#=
下面代码生成 myshow 宏体
=#

macro myshow(exs...)
    blk = Expr(:block)  #此处使用 头部符号是 block的表达式
    for ex in exs
        push!(blk.args, :(println($(sprint(Base.show_unquoted,ex)*" = "))))
        push!(blk.args, :(show(stdout, "text/plain", begin value=$(esc(ex)) end)))
        push!(blk.args, :(println()))
    end
    isempty(exs) || push!(blk.args, :value)
    return blk
end


@myshow (macro with 1 method)

In [3]:
#=
下面代码块生成D矩阵=K2+I2
=#
using LinearAlgebra  #引入包裹
n=2  #T=3期
#v = fill(-1, n)  #元素为 -1 的向量
#K = Diagonal(v)   #此处其实是负的单位矩阵，所以代码优化为下面 K=-I
w=ones(n)
I = Diagonal(w)
K=-I
u=zeros(n)
K2=hcat(K,u)
I2=hcat(u,I)
D=K2+I2  
V=D*D'
@myshow(K2, I2, D, V)

K2 = 
2×3 Matrix{Float64}:
 -1.0   0.0  0.0
  0.0  -1.0  0.0
I2 = 
2×3 Matrix{Float64}:
 0.0  1.0  0.0
 0.0  0.0  1.0
D = 
2×3 Matrix{Float64}:
 -1.0   1.0  0.0
  0.0  -1.0  1.0
V = 
2×2 Matrix{Float64}:
  2.0  -1.0
 -1.0   2.0


2×2 Matrix{Float64}:
  2.0  -1.0
 -1.0   2.0

In [4]:
#=
下面的代码块生成V和V-1的乔列斯基分解
=#
L1=cholesky(V)
V1=inv(V)
B1=cholesky(V1)

@myshow(L1, B1)

L1 = 
Cholesky{Float64, Matrix{Float64}}
U factor:
2×2 UpperTriangular{Float64, Matrix{Float64}}:
 1.41421  -0.707107
  ⋅        1.22474
B1 = 
Cholesky{Float64, Matrix{Float64}}
U factor:
2×2 UpperTriangular{Float64, Matrix{Float64}}:
 0.816497  0.408248
  ⋅        0.707107


Cholesky{Float64, Matrix{Float64}}
U factor:
2×2 UpperTriangular{Float64, Matrix{Float64}}:
 0.816497  0.408248
  ⋅        0.707107

In [5]:
#=
检验乔列斯基分解
V1表示逆矩阵, L1表示L的转置  在 不至于引起混淆的情况下，用1表示转置或者逆矩阵
V=LL', V-1=BB'
=#

L1 = [2^0.5  -1/2^0.5; 0  3/6^0.5]
B1= [2/6^0.5  1/6^0.5; 0  1/2^0.5]
L=L1'
B=B1'
V=L*L'
V1=B*B'
I0=V*V1
@myshow(V, V1, I0)

V = 
2×2 Matrix{Float64}:
  2.0  -1.0
 -1.0   2.0
V1 = 
2×2 Matrix{Float64}:
 0.666667  0.333333
 0.333333  0.666667
I0 = 
2×2 Matrix{Float64}:
  1.0          1.11022e-16
 -1.11022e-16  1.0


2×2 Matrix{Float64}:
  1.0          1.11022e-16
 -1.11022e-16  1.0

In [6]:
#=
下面的代码生成A+矩阵 A=H2=(DD')^-0.5 * D =B' * D = R * A+ =A2
=#

A=B'*D
r=[(2/3)^0.5;(1/2)^0.5]
R=Diagonal(r)

I20=hcat(I,u)
A2=[-0.5   -0.5; 0  -1]
A20=hcat(u,A2)
Ap=I20 + A20
A2=R*Ap
@myshow(A, Ap, A2)

A = 
2×3 Matrix{Float64}:
 -0.816497   0.408248  0.408248
  0.0       -0.707107  0.707107
Ap = 
2×3 Matrix{Float64}:
 1.0  -0.5  -0.5
 0.0   1.0  -1.0
A2 = 
2×3 Matrix{Float64}:
 0.816497  -0.408248  -0.408248
 0.0        0.707107  -0.707107


2×3 Matrix{Float64}:
 0.816497  -0.408248  -0.408248
 0.0        0.707107  -0.707107

In [7]:
#=
下面演示后向正交离差
=#
r2=[(1/2)^0.5; (2/3)^0.5]
R2=Diagonal(r2)

#I2=hcat(u,I)
A3=[-1  0; -0.5   -0.5]
A30=hcat(A3, u)
Am=I2 + A30   #Ap, p=plus + , Am, m=minus -
A3=R2*Am
@myshow(A, Am, A3)

A = 
2×3 Matrix{Float64}:
 -0.816497   0.408248  0.408248
  0.0       -0.707107  0.707107
Am = 
2×3 Matrix{Float64}:
 -1.0   1.0  0.0
 -0.5  -0.5  1.0
A3 = 
2×3 Matrix{Float64}:
 -0.707107   0.707107  0.0
 -0.408248  -0.408248  0.816497


2×3 Matrix{Float64}:
 -0.707107   0.707107  0.0
 -0.408248  -0.408248  0.816497

In [8]:
#=
根据交叉对应。V-1=TT', V=LL', T=(L-1)', T'=L-1
=#
T1=inv(L)
A4=T1*D
@myshow(T1, A3, A4)

T1 = 
2×2 adjoint(::Matrix{Float64}) with eltype Float64:
 0.707107  0.0
 0.408248  0.816497
A3 = 
2×3 Matrix{Float64}:
 -0.707107   0.707107  0.0
 -0.408248  -0.408248  0.816497
A4 = 
2×3 Matrix{Float64}:
 -0.707107   0.707107  0.0
 -0.408248  -0.408248  0.816497


2×3 Matrix{Float64}:
 -0.707107   0.707107  0.0
 -0.408248  -0.408248  0.816497

In [ ]:
#last